# Compute time of each classifiers for each dataset and each cell types.

In [1]:
lapply(c("dplyr","Seurat","HGNChelper", "adverSCarial", "IRdisplay"), library, character.only = T)
options(warn=-1)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
 [1] "Seurat"       "SeuratObject" "sp"           "dplyr"        "stats"       
 [6] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[11] "base"        

[[3]]
 [1] "HGNChelper"   "Seurat"       "SeuratObject" "sp"           "dplyr"       
 [6] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[11] "methods"      "base"        

[[4]]
 [1] "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject" "sp"          
 [6] "dplyr"        "stats"        "graphics"     "grDevices"    "utils"       
[11] "datasets"     "methods"      "base"        

[[5]]
 [1] "IRdisplay"    "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject"
 [6] "sp"           "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"

In [2]:
bd <- "data/"

my_computer= "c1"

source("load_classifiers_v5.r")

In [3]:
c_paths <- list(pbmc_3k=list(so=paste0(bd, "v2//pbmc_test.rds"),
                             sce=paste0(bd, "v2//sce_pbmc_test.rds"),
                             df=paste0(bd, "v2//seurat_scaled_pbmc_test.txt")),
               kidney_10k=list(so=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_kidney_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.txt")),
                liver_10k=list(so=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_liver_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.txt")),
                axilla_10k=list(so=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_axilla_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.txt"))
               )

In [4]:
lsg <- list(pbmc_3k= read.table("data//v2/sign_genes.txt")$gene,
           liver_10k = read.table("data//v5/data/sign/sign_liver_10k.txt")$gene,
           kidney_10k = read.table("data//v5/data/sign/sign_kidney_10k.txt")$gene,
           axilla_10k = read.table("data//v5/data/sign/sign_axilla_10k.txt")$gene)

In [5]:
c_corres <- readRDS("data//v5/type_corres.rds")

In [6]:
ct_pbmc <- unique(unlist(lapply(grep("pbmc",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_pbmc

[1] "Memory CD4 T" "B"            "CD14+ Mono"   "NK"           "CD8 T"       
[6] "Naive CD4 T"  "FCGR3A+ Mono" "DC"           "Platelet"

In [7]:
ct_axilla <- unique(unlist(lapply(grep("axilla",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_axilla

[1] "malignant cell"                  "blood vessel endothelial cell"  
[3] "fibroblast"                      "macrophage"                     
[5] "blood vessel smooth muscle cell" "T cell"

In [8]:
ct_liver <- unique(unlist(lapply(grep("liver",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_liver

[1] "malignant cell"                  "endothelial cell"               
[3] "blood vessel smooth muscle cell" "macrophage"                     
[5] "monocyte"                        "mature NK T cell"

In [9]:
ct_kidney <- unique(unlist(lapply(grep("kidney",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_kidney

[1] "kidney epithelial cell"              
[2] "kidney cell"                         
[3] "podocyte"                            
[4] "epithelial cell of proximal tubule"  
[5] "mesenchymal cell"                    
[6] "kidney loop of Henle epithelial cell"

In [10]:
l_cts <- list("pbmc_3k"=ct_pbmc, "kidney_10k"=ct_kidney,
              "liver_10k"=ct_liver, "axilla_10k"=ct_axilla)

In [11]:
c_ids2run = c()
for (str_class in c("scType", "CHETAH", "scAnnotatR", "RF", "MLP")){
    for (str_ds in names(l_cts)){
        for (str_ct in l_cts[[str_ds]]){
            c_ids2run <- c(c_ids2run, paste0(str_class,";",str_ds,";",str_ct))
        }
    }
}
c_ids2run <- c(paste0("c1;",c_ids2run), paste0("c2;", c_ids2run[length(c_ids2run):1]))
# c_ids2run <- c(paste0("c1;",c_ids2run), paste0("c2;", c_ids2run))
c_ids2run

[1] "c1;scType;pbmc_3k;Memory CD4 T"                               
  [2] "c1;scType;pbmc_3k;B"                                          
  [3] "c1;scType;pbmc_3k;CD14+ Mono"                                 
  [4] "c1;scType;pbmc_3k;NK"                                         
  [5] "c1;scType;pbmc_3k;CD8 T"                                      
  [6] "c1;scType;pbmc_3k;Naive CD4 T"                                
  [7] "c1;scType;pbmc_3k;FCGR3A+ Mono"                               
  [8] "c1;scType;pbmc_3k;DC"                                         
  [9] "c1;scType;pbmc_3k;Platelet"                                   
 [10] "c1;scType;kidney_10k;kidney epithelial cell"                  
 [11] "c1;scType;kidney_10k;kidney cell"                             
 [12] "c1;scType;kidney_10k;podocyte"                                
 [13] "c1;scType;kidney_10k;epithelial cell of proximal tubule"      
 [14] "c1;scType;kidney_10k;mesenchymal cell"                        
 [15] "c1;scType;kidney_10k;kidney loop of Henle epithelial cell"    
 [16] "c1;scType;liver_10k;malignant cell"                           
 [17] "c1;scType;liver_10k;endothelial cell"                         
 [18] "c1;scType;liver_10k;blood vessel smooth muscle cell"          
 [19] "c1;scType;liver_10k;macrophage"                               
 [20] "c1;scType;liver_10k;monocyte"                                 
 [21] "c1;scType;liver_10k;mature NK T cell"                         
 [22] "c1;scType;axilla_10k;malignant cell"                          
 [23] "c1;scType;axilla_10k;blood vessel endothelial cell"           
 [24] "c1;scType;axilla_10k;fibroblast"                              
 [25] "c1;scType;axilla_10k;macrophage"                              
 [26] "c1;scType;axilla_10k;blood vessel smooth muscle cell"         
 [27] "c1;scType;axilla_10k;T cell"                                  
 [28] "c1;CHETAH;pbmc_3k;Memory CD4 T"                               
 [29] "c1;CHETAH;pbmc_3k;B"                                          
 [30] "c1;CHETAH;pbmc_3k;CD14+ Mono"                                 
 [31] "c1;CHETAH;pbmc_3k;NK"                                         
 [32] "c1;CHETAH;pbmc_3k;CD8 T"                                      
 [33] "c1;CHETAH;pbmc_3k;Naive CD4 T"                                
 [34] "c1;CHETAH;pbmc_3k;FCGR3A+ Mono"                               
 [35] "c1;CHETAH;pbmc_3k;DC"                                         
 [36] "c1;CHETAH;pbmc_3k;Platelet"                                   
 [37] "c1;CHETAH;kidney_10k;kidney epithelial cell"                  
 [38] "c1;CHETAH;kidney_10k;kidney cell"                             
 [39] "c1;CHETAH;kidney_10k;podocyte"                                
 [40] "c1;CHETAH;kidney_10k;epithelial cell of proximal tubule"      
 [41] "c1;CHETAH;kidney_10k;mesenchymal cell"                        
 [42] "c1;CHETAH;kidney_10k;kidney loop of Henle epithelial cell"    
 [43] "c1;CHETAH;liver_10k;malignant cell"                           
 [44] "c1;CHETAH;liver_10k;endothelial cell"                         
 [45] "c1;CHETAH;liver_10k;blood vessel smooth muscle cell"          
 [46] "c1;CHETAH;liver_10k;macrophage"                               
 [47] "c1;CHETAH;liver_10k;monocyte"                                 
 [48] "c1;CHETAH;liver_10k;mature NK T cell"                         
 [49] "c1;CHETAH;axilla_10k;malignant cell"                          
 [50] "c1;CHETAH;axilla_10k;blood vessel endothelial cell"           
 [51] "c1;CHETAH;axilla_10k;fibroblast"                              
 [52] "c1;CHETAH;axilla_10k;macrophage"                              
 [53] "c1;CHETAH;axilla_10k;blood vessel smooth muscle cell"         
 [54] "c1;CHETAH;axilla_10k;T cell"                                  
 [55] "c1;scAnnotatR;pbmc_3k;Memory CD4 T"                           
 [56] "c1;scAnnotatR;pbmc_3k;B"                                      
 [57] "c1;scAnnotatR;pbmc_3k;CD14+ Mono"                             
 [58] "c1;sc

In [12]:
df_times <- readRDS("data//v5/df_simple_times.rds")
df_times

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;Memory CD4 T,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;B,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;CD14+ Mono,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;NK,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;CD8 T,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;Naive CD4 T,-1,-1,-1,-1,-1,-1
c1;scType;pbmc_3k;FCGR3A+ Mono,-1,-1,-1,-1,-1,-1


In [13]:
df_times[df_times$run_time != -1,]


,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
c2;MLP;axilla_10k;T cell,-1,-1,-1,-1,33.016447,-1
c2;MLP;axilla_10k;blood vessel smooth muscle cell,-1,-1,-1,-1,194.882024,-1
c2;MLP;axilla_10k;macrophage,-1,-1,-1,-1,156.780317,-1
c2;MLP;axilla_10k;fibroblast,-1,-1,-1,-1,192.994326,-1
c2;MLP;axilla_10k;blood vessel endothelial cell,-1,-1,-1,-1,316.758781,-1
c2;MLP;axilla_10k;malignant cell,-1,-1,-1,-1,213.884771,-1
c2;MLP;liver_10k;mature NK T cell,-1,-1,-1,-1,106.518395,-1
c2;MLP;liver_10k;monocyte,-1,-1,-1,-1,209.386986,-1
c2;MLP;liver_10k;macrophage,-1,-1,-1,-1,175.024044,-1


In [14]:
df_times['c2;MLP;axilla_10k;T cell',]

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
c2;MLP;axilla_10k;T cell,-1,-1,-1,-1,33.01645,-1


In [15]:
myHourFormat <- function(start_time){
    diff_secs <- as.numeric(difftime(Sys.time(),start_time, units = "secs"))

    # Extraction des heures, minutes et secondes
    heures <- floor(diff_secs / 3600)
    minutes <- floor((diff_secs %% 3600) / 60)
    secondes <- round(diff_secs %% 60)

    # Formatage du temps écoulé
    temps_formate <- sprintf("%02d:%02d:%02d", heures, minutes, secondes)
    return(temps_formate)    
}


In [16]:
c_ids2run[1:10]

[1] "c1;scType;pbmc_3k;Memory CD4 T"             
 [2] "c1;scType;pbmc_3k;B"                        
 [3] "c1;scType;pbmc_3k;CD14+ Mono"               
 [4] "c1;scType;pbmc_3k;NK"                       
 [5] "c1;scType;pbmc_3k;CD8 T"                    
 [6] "c1;scType;pbmc_3k;Naive CD4 T"              
 [7] "c1;scType;pbmc_3k;FCGR3A+ Mono"             
 [8] "c1;scType;pbmc_3k;DC"                       
 [9] "c1;scType;pbmc_3k;Platelet"                 
[10] "c1;scType;kidney_10k;kidney epithelial cell"

In [1]:
only_myComp_ids <- grep(paste0("^",my_computer), c_ids2run, value=T)
for (str_id in only_myComp_ids){
    display(str_id)
    a_a = unlist(strsplit(str_id, ";"))
#     my_comp <- a_a[1]

    baseClass <- a_a[2]
    basen <- a_a[3]
    myCluster <- a_a[4]
    
    if (!str_id %in% rownames(df_times)){
        df_times[str_id,] = -1
    }
    if (nchar(as.character(df_times[str_id,"run_time"])) == 2){
        display(str_id)
# for (a_a in c_atts){
#     basen = a_a[1]
#     baseClass = a_a[2]
#     modif = a_a[3]
#     myCluster = a_a[4]
# for (basen in c_b2){
        message(basen)
        my_path <- paste0(bd, "v5/data/sc/hgnc_", basen, "_test.txt")
        my_path <- c_paths[[basen]]$df
        if (!file.exists(my_path)){
            message(my_path, " not available")
            next
        }
        df_pbmc_test <- read.table(my_path)
        expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
        clusters_df <- df_pbmc_test$y
        names(clusters_df) <- rownames(df_pbmc_test)

#         sign_path <- paste0(paste0(bd, "v5/data/sign/sign_", basen, ".txt"))
#         if (!file.exists(sign_path)){
#             message(sign_path, " not available, computing getSignGenes")
#             dfSG <- getSignGenes(expr_df, clusters_df, verbose=T)
#             write.table(dfSG$results, sign_path)
#         }
#         mySignGenes <- read.table(sign_path)

        message(baseClass)
        strClassifier <- paste0(baseClass, "_", basen)
        if ( baseClass == "scType" || basen=="pbmc_3k"){
            strClassifier <- baseClass
        }
        display(strClassifier)
        myClassifier <- classifList[[strClassifier]]$fct
        inputType <- classifList[[strClassifier]]$inputType

        display(inputType)
        mySlot <- classifList[[strClassifier]]$slot
        expr <- ""

        if(inputType == "data.frame"){
            expr <- expr_df
            clusters <- clusters_df
        }
        if(tolower(inputType) == "seurat"){
            my_path <- paste0(bd, "v5/data/sc/hgnc_", basen, "_test.rds")
            my_path <- c_paths[[basen]]$so
            if (!file.exists(my_path)){
                message(my_path, " not available")
                break
            }
            so_pbmc_test <- readRDS(my_path)

            clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_so) <- rownames(so_pbmc_test@meta.data)

            expr <- so_pbmc_test
            clusters <- clusters_so
        }
        if (tolower(inputType) == "singlecellexperiment"){
            my_path1 <- paste0(bd, "v5/data/sc/hgnc_",basen,"_test.rds")
            my_path1 <- c_paths[[basen]]$so
            my_path2 <- paste0(bd, "v5/data/sc/sce_hgnc_",basen,"_test.rds")
            my_path2 <- c_paths[[basen]]$sce
            if (!file.exists(my_path1) || !file.exists(my_path2)){
                message(my_path1, " or ", my_path2," not available")
                break
            }
            so_pbmc_test <- readRDS(my_path1)
            clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_so) <- rownames(so_pbmc_test@meta.data)

            sce_pbmc_test <- readRDS(my_path2)
            clusters_sce = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_sce) <- rownames(so_pbmc_test@meta.data)

            expr <- sce_pbmc_test
            clusters <- clusters_sce
        }
        message(myCluster)
        my_clusters = as.character(unname(clusters))
        if ( inputType == "SingleCellExperiment"){
            my_clusters = as.character(expr@colData$chr_seurat_cluster)
        }

        t1 <- Sys.time()
        class_res <- myClassifier(expr, my_clusters, myCluster)
        display(class_res)
        new_time <- difftime(Sys.time(), t1, units = "secs")
        df_times <- readRDS("data//v5/df_simple_times.rds")
        df_times[str_id, "run_time"] <- new_time
        saveRDS(df_times, "data//v5/df_simple_times.rds")
    } else {
        display(paste(str_id, " already done"))
    }
}


ERROR: Error in eval(expr, envir, enclos): object 'my_computer' not found


In [18]:
df_temp <- df_times[grep("c2", rownames(df_times), value=T),]
mean(df_temp[grep("MLP", rownames(df_temp)), "run_time"])

[1] 138.3845

In [19]:
df_temp <- df_times[grep("c2", rownames(df_times), value=T),]
mean(df_temp[grep("scType", rownames(df_temp)), "run_time"])

[1] 500.7169

In [20]:
df_temp <- df_times[grep("c2", rownames(df_times), value=T),]
mean(df_temp[grep("RF", rownames(df_temp)), "run_time"])

[1] 220.9521

In [21]:
df_temp <- df_times[grep("c2", rownames(df_times), value=T),]
mean(df_temp[grep("scAnnotatR", rownames(df_temp)), "run_time"])

[1] 513.9234

In [22]:
df_temp <- df_times[grep("c2", rownames(df_times), value=T),]
mean(df_temp[grep("CHETAH", rownames(df_temp)), "run_time"])

[1] 254.6231

In [25]:
head(my_clusters)

[1] "malignant cell" "malignant cell" "malignant cell" "malignant cell"
[5] "malignant cell" "malignant cell"

In [26]:
myCluster

[1] "T cell"

In [28]:
myClassifier(expr, my_clusters, myCluster)

,HTAPP-878-SMP-7149-TST-channel1_GATGTTGCAAACGTGG-1,HTAPP-878-SMP-7149-TST-channel1_CTTTCAAGTAGGTACG-1,HTAPP-878-SMP-7149-TST-channel1_CACCGTTGTTCTGACA-1,HTAPP-878-SMP-7149-TST-channel1_GAGTTTGCACAACGTT-1,HTAPP-878-SMP-7149-TST-channel1_GATGACTTCTTTGCTA-1,HTAPP-878-SMP-7149-TST-channel1_CCCTAACCAAGCACCC-1,HTAPP-878-SMP-7149-TST-channel1_GTCACGGCATTACTCT-1,HTAPP-878-SMP-7149-TST-channel1_CATACCCTCGCTGCGA-1,HTAPP-878-SMP-7149-TST-channel1_ACTATTCTCAAGTCGT-1,HTAPP-878-SMP-7149-TST-channel1_TCACTATTCTGGTTGA-1,⋯,HTAPP-878-SMP-7149-TST-channel2_CTGATCCAGACTGGGT-1,HTAPP-878-SMP-7149-TST-channel2_CCATAAGCAAGCACAG-1,HTAPP-878-SMP-7149-TST-channel2_GTGGAGACAAATCAGA-1,HTAPP-878-SMP-7149-TST-channel2_GGAACCCGTCGGAAAC-1,HTAPP-878-SMP-7149-TST-channel2_CCCTTAGGTCTCAGGC-1,HTAPP-878-SMP-7149-TST-channel2_GCGATCGGTACTCGCG-1,HTAPP-878-SMP-7149-TST-channel2_TCACGGGCAGCTTTCC-1,HTAPP-878-SMP-7149-TST-channel2_CTCCACAGTGTCATTG-1,HTAPP-878-SMP-7149-TST-channel2_CATGAGTAGCTAAATG-1,HTAPP-878-SMP-7149-TST-channel2_GGTTCTCGTGGCTCTG-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
blood vessel endothelial cell,2.961845e-21,3.226691e-19,1.697217e-26,4.006229e-25,1.380018e-25,8.584628e-33,5.308286e-31,1.243128e-25,9.295548e-27,4.325117e-30,⋯,1.947621e-06,2.166200e-10,5.061348e-07,1.486810e-07,1.061834e-05,5.687372e-08,4.028705e-05,2.724211e-07,7.375722e-06,3.827095e-06
blood vessel smooth muscle cell,3.215563e-24,7.448658e-22,4.302244e-30,1.432534e-28,2.952323e-29,1.204865e-36,2.874040e-35,2.990431e-29,6.440495e-31,1.557924e-33,⋯,4.366187e-08,1.446303e-16,3.290917e-09,1.668707e-10,3.534787e-07,4.055749e-10,1.030190e-06,1.386540e-09,2.161821e-07,2.530743e-08
fibroblast,7.702430e-27,6.385518e-26,2.809415e-33,3.601892e-31,5.312436e-32,0.000000e+00,2.363739e-37,6.175476e-32,1.495374e-33,7.410454e-37,⋯,7.390655e-06,8.111646e-13,2.196957e-06,3.192116e-08,3.796818e-05,4.726523e-08,4.990849e-04,9.066648e-08,1.403248e-05,3.224936e-06
macrophage,1.922037e-29,3.273949e-30,1.888648e-36,1.591568e-34,9.483943e-36,0.000000e+00,0.000000e+00,1.183677e-35,5.291610e-37,0.000000e+00,⋯,9.999905e-01,9.999999e-01,9.999972e-01,9.999997e-01,9.999507e-01,9.999998e-01,9.994591e-01,9.999996e-01,9.999784e-01,9.999929e-01
malignant cell,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.999999e-01,⋯,3.118178e-08,1.492625e-14,2.251709e-09,5.659945e-10,2.526405e-07,3.980551e-11,3.535611e-07,5.546396e-10,5.103583e-08,2.253321e-08
T cell,4.736470e-26,4.709301e-26,7.700144e-33,5.106331e-31,2.991213e-32,0.000000e+00,0.000000e+00,6.726624e-32,2.798014e-33,1.746080e-37,⋯,2.054280e-10,9.514873e-17,3.555644e-12,1.341524e-11,1.830734e-09,2.692994e-12,1.001856e-09,3.301521e-11,1.317815e-09,1.442325e-09


In [29]:
df_times[grep('c2;scAnnotatR', rownames(df_times), value=T),]

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
c2;scAnnotatR;axilla_10k;T cell,-1,-1,-1,-1,498.839635,-1
c2;scAnnotatR;axilla_10k;blood vessel smooth muscle cell,-1,-1,-1,-1,867.754949,-1
c2;scAnnotatR;axilla_10k;macrophage,-1,-1,-1,-1,891.484807,-1
c2;scAnnotatR;axilla_10k;fibroblast,-1,-1,-1,-1,959.829151,-1
c2;scAnnotatR;axilla_10k;blood vessel endothelial cell,-1,-1,-1,-1,857.493401,-1
c2;scAnnotatR;axilla_10k;malignant cell,-1,-1,-1,-1,874.980423,-1
c2;scAnnotatR;liver_10k;mature NK T cell,-1,-1,-1,-1,580.831894,-1
c2;scAnnotatR;liver_10k;monocyte,-1,-1,-1,-1,332.876665,-1
c2;scAnnotatR;liver_10k;macrophage,-1,-1,-1,-1,618.532580,-1
